In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

# 공구 일정 페이지 csv / 친스매출 최근 1년
df = pd.read_csv('/Users/mac/Downloads/[공구] 일정 신청 및 관리 시트 - 공구전체일정 (2).csv')
fs_df = pd.read_excel('/Users/mac/Downloads/주문내역_2024-06-02~2025-06-02.xlsx')

# pb_user 테이블, auth_info 테이블, excel_group_buy_info 테이블 25년 1월~
# t_user 테이블은 전체
# excel_group_buy_info은 데이터가 커서 5만개 내로 쿼리 조정
auth_df = pd.read_csv('/Users/mac/Downloads/query_1-2025-06-03_113935.csv')
user_df = pd.read_csv('/Users/mac/Downloads/query_2-2025-06-03_114007.csv')
sales_df = pd.read_csv('/Users/mac/Downloads/query_3-2025-06-03_114057.csv')
t_user_df = pd.read_csv('/Users/mac/Downloads/store_t_user_export_2025-06-03_234130.csv')

# 친스 판매중, 판매종료된 공구 상품번호 입력
gongu = [51276,
         51275,
         51274,
         51273,
         51272,
         51271,
         51269,
         51268,
         51267,
         51266,
         51262,
         51260,
         51259,
         51258,
         51257,
         51256,
         51255,
         51254,
         51253,
         51182,
         51181,
         51178,
         51177,
         51176,
         51174,
         50557]

In [2]:
status = ['배송중', '배송완료', '구매확정', '배송준비', '결제완료']

fs_df = fs_df[fs_df['주문 상태'].isin(status)]
fs_gongu_df = fs_df[fs_df['상품 번호'].isin(gongu)]

In [3]:
df.columns = df.loc[1]
df = df.drop([0,1])
con1 = df['진행상황-업체'] == '업체-일정확정'
con2 = df['진행상황-업체'] == '업체-공구종료'
df = df[con1 | con2]
df = df.reset_index()
df.loc[df['시작일'].str.len() <= 8, '시작일'] = '25/' + df['시작일']
df = df[df['시작일'].isna() == False]

In [4]:
df.drop(df[df['시작일'] == '약사님 일정 조율중'].index, inplace=True)

In [5]:
dates = []

for i in df['시작일']:
    date = i.split('(')[0].strip()
    year = '20' + date.split('/')[0].strip()
    month = date.split('/')[1].strip()
    day = date.split('/')[2].strip()
    final = year + '-' + month + '-' + day
    dates.append(final)

df['date'] = dates
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= datetime(2025, 2, 17)]

sales_df['time'] = pd.to_datetime(sales_df['reg_dt']) + timedelta(hours=9)
sales_df['minute'] = sales_df['time'].dt.strftime('%H:%M:%S')
sales_df['minute'] = pd.to_datetime(sales_df['minute'])
sales_df['minute'].mean()

Timestamp('2025-06-03 11:21:00.910303488')

In [6]:
print('2월 4주차 부터 매주 예정된 공구건은')
df.groupby(pd.Grouper(key='date', freq='W'))['진행상황-업체'].count()[5:20]

2월 4주차 부터 매주 예정된 공구건은


date
2025-03-30    35
2025-04-06    47
2025-04-13    44
2025-04-20    37
2025-04-27    35
2025-05-04    36
2025-05-11    28
2025-05-18    35
2025-05-25    37
2025-06-01    39
2025-06-08    28
2025-06-15    40
2025-06-22    42
2025-06-29    34
2025-07-06    31
Freq: W-SUN, Name: 진행상황-업체, dtype: int64

In [7]:
print('주간 친한스토어 주간 공구 진행 건수')
datas = {}

for i in fs_gongu_df['상품명'].unique():
    con = fs_gongu_df['상품명'] == i
    value = fs_gongu_df.loc[con, '결제 완료일'].min()
    key = i
    datas[key] = value

temp_df = pd.DataFrame.from_dict(datas, orient="index", columns=['date'])
print(temp_df.groupby(pd.Grouper(key='date', freq='W')).size())

주간 친한스토어 주간 공구 진행 건수
date
2025-02-23    1
2025-03-02    0
2025-03-09    0
2025-03-16    0
2025-03-23    5
2025-03-30    1
2025-04-06    0
2025-04-13    5
2025-04-20    3
2025-04-27    4
2025-05-04    1
2025-05-11    4
Freq: W-SUN, dtype: int64


In [8]:
print('주간 파마브로스 전체 공구 매출')
sales_df.groupby(pd.Grouper(key='time', freq='W'))['order_price'].sum()

주간 파마브로스 전체 공구 매출


time
2025-05-11     909602100
2025-05-18    1216797800
2025-05-25    1349455360
2025-06-01    1539659020
2025-06-08     437846500
Freq: W-SUN, Name: order_price, dtype: int64

In [9]:
print('매주 친한스토어 공동구매 매출은')
fs_gongu_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['정산가'].sum()

매주 친한스토어 공동구매 매출은


결제 완료일
2025-02-23    11570400
2025-03-02     4608000
2025-03-09           0
2025-03-16           0
2025-03-23    62824300
2025-03-30    33188200
2025-04-06     3813100
2025-04-13    53078300
2025-04-20    56088350
2025-04-27    64174750
2025-05-04    11777100
2025-05-11    44815000
2025-05-18    23645250
Freq: W-SUN, Name: 정산가, dtype: int64

In [10]:
print('주간 파마브로스 전체 공구 건수')
sales_df.groupby(pd.Grouper(key='time', freq='W'))['order_no'].count()

주간 파마브로스 전체 공구 건수


time
2025-05-11     8493
2025-05-18    11720
2025-05-25    11450
2025-06-01    13519
2025-06-08     3995
Freq: W-SUN, Name: order_no, dtype: int64

In [11]:
print('매주 친한스토어 공구 매출 건수는')
fs_gongu_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['결제 번호'].size()

매주 친한스토어 공구 매출 건수는


결제 완료일
2025-02-23     71
2025-03-02     28
2025-03-09      0
2025-03-16      0
2025-03-23    488
2025-03-30    271
2025-04-06     35
2025-04-13    405
2025-04-20    467
2025-04-27    441
2025-05-04     87
2025-05-11    308
2025-05-18    151
Freq: W-SUN, Name: 결제 번호, dtype: int64

In [12]:
print('매주 친한스토어 일반 매출은')
fs_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['정산가'].sum()

매주 친한스토어 일반 매출은


결제 완료일
2024-09-22       14000
2024-09-29      205660
2024-10-06      763300
2024-10-13      407070
2024-10-20     1081060
2024-10-27     1177000
2024-11-03     1700250
2024-11-10      183350
2024-11-17       42750
2024-11-24      213700
2024-12-01       73400
2024-12-08           0
2024-12-15       42750
2024-12-22      188300
2024-12-29      105600
2025-01-05       34900
2025-01-12       17400
2025-01-19           0
2025-01-26      159600
2025-02-02      117000
2025-02-09       53500
2025-02-16           0
2025-02-23    11608900
2025-03-02     4712200
2025-03-09       57700
2025-03-16         600
2025-03-23    62880350
2025-03-30    34114080
2025-04-06     5376760
2025-04-13    55443600
2025-04-20    58516690
2025-04-27    66326380
2025-05-04    13491620
2025-05-11    46545730
2025-05-18    26080590
2025-05-25     1385570
2025-06-01     2133390
2025-06-08     3574050
Freq: W-SUN, Name: 정산가, dtype: int64

In [13]:
## merged_df = user_df.merge(auth_df, left_on='id', right_on='user_id', how='inner')
merged_df = user_df.merge(auth_df, left_on='id', right_on='user_id', how='inner')
merged_df['reg_dt'] = pd.to_datetime(merged_df['reg_dt'], format='mixed')
print('매주 친한스토어 친약+친스 회원가입자 수는')
merged_df.groupby(pd.Grouper(key='reg_dt', freq='W')).size()[5:]

매주 친한스토어 친약+친스 회원가입자 수는


reg_dt
2025-02-09     397
2025-02-16     337
2025-02-23     612
2025-03-02     557
2025-03-09     614
2025-03-16     471
2025-03-23    1133
2025-03-30     809
2025-04-06     472
2025-04-13    1040
2025-04-20    1068
2025-04-27    1014
2025-05-04     536
2025-05-11     868
2025-05-18     637
2025-05-25     416
2025-06-01     368
2025-06-08     267
Freq: W-SUN, dtype: int64

In [14]:
print('친스 회원가입자 수는 (t_user 중에서 di 없는 유저만)')
t_user_df['date'] = t_user_df['created_at'].apply(lambda x: f'{x[:10]}')
t_user_df['date'] = pd.to_datetime(t_user_df['date'])

temp_df = t_user_df[t_user_df['date'] >= datetime(2025, 1, 1)]
merge_df = pd.merge(temp_df, auth_df[['id', 'ci', 'di', 'user_id']], left_on='pbuser_id', right_on='user_id', how='left')
merge_df = merge_df[merge_df['di'].isna()]
merge_df.groupby(pd.Grouper(key='date', freq='W')).size()

친스 회원가입자 수는 (t_user 중에서 di 없는 유저만)


date
2025-01-12      1
2025-01-19      1
2025-01-26      2
2025-02-02      0
2025-02-09     20
2025-02-16     50
2025-02-23    132
2025-03-02     32
2025-03-09     30
2025-03-16      8
2025-03-23    702
2025-03-30    430
2025-04-06     60
2025-04-13    573
2025-04-20    642
2025-04-27    579
2025-05-04    152
2025-05-11    367
2025-05-18    182
2025-05-25     21
2025-06-01     27
2025-06-08    201
Freq: W-SUN, dtype: int64

In [15]:
print('친스 회원가입자 수는 (di 없는 유저만)')
auth_df_di = auth_df[auth_df['di'].isna()]
auth_df_di['created_at'] = pd.to_datetime(auth_df_di['created_at'], format='mixed')
auth_df_di.groupby(pd.Grouper(key='created_at', freq='W')).size()

친스 회원가입자 수는 (di 없는 유저만)


created_at
2025-02-23    105
2025-03-02     23
2025-03-09      0
2025-03-16      0
2025-03-23    632
2025-03-30    369
2025-04-06     46
2025-04-13    517
2025-04-20    592
2025-04-27    539
2025-05-04    141
2025-05-11    340
2025-05-18    156
2025-05-25     24
2025-06-01     24
2025-06-08    188
Freq: W-SUN, dtype: int64